In [2]:
import numpy as np
from tensorflow.keras.utils import to_categorical
with open("male.txt", "r") as file:
    names = file.read().splitlines()

all_names = ''.join(names)

chars = sorted(set(all_names))
char_to_int = {char: idx for idx, char in enumerate(chars)}
int_to_char = {idx: char for idx, char in enumerate(chars)}

sequence_length = 10
X = []
y = []

for i in range(len(all_names) - sequence_length):
    X.append([char_to_int[char] for char in all_names[i:i+sequence_length]])
    y.append(char_to_int[all_names[i + sequence_length]])

X = np.reshape(X, (len(X), sequence_length, 1)) / float(len(chars))
y = to_categorical(y, num_classes=len(chars))


In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

model = Sequential()
model.add(LSTM(128, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(LSTM(128))
model.add(Dense(len(chars), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [4]:
model.fit(X, y, epochs=50, batch_size=128)


Epoch 1/50
137/137 ━━━━━━━━━━━━━━━━━━━━ 13s 71ms/step - loss: 3.5836
Epoch 2/50
137/137 ━━━━━━━━━━━━━━━━━━━━ 10s 68ms/step - loss: 3.3474
Epoch 3/50
137/137 ━━━━━━━━━━━━━━━━━━━━ 9s 57ms/step - loss: 3.3462
Epoch 4/50
137/137 ━━━━━━━━━━━━━━━━━━━━ 10s 74ms/step - loss: 3.3495
Epoch 5/50
137/137 ━━━━━━━━━━━━━━━━━━━━ 9s 68ms/step - loss: 3.3399
Epoch 6/50
137/137 ━━━━━━━━━━━━━━━━━━━━ 8s 57ms/step - loss: 3.3304
Epoch 7/50
137/137 ━━━━━━━━━━━━━━━━━━━━ 9s 69ms/step - loss: 3.2685
Epoch 8/50
137/137 ━━━━━━━━━━━━━━━━━━━━ 10s 69ms/step - loss: 3.1969
Epoch 9/50
137/137 ━━━━━━━━━━━━━━━━━━━━ 8s 57ms/step - loss: 3.1211
Epoch 10/50
137/137 ━━━━━━━━━━━━━━━━━━━━ 10s 59ms/step - loss: 3.0306
Epoch 11/50
137/137 ━━━━━━━━━━━━━━━━━━━━ 11s 64ms/step - loss: 2.9512
Epoch 12/50
137/137 ━━━━━━━━━━━━━━━━━━━━ 9s 68ms/step - loss: 2.8944
Epoch 13/50
137/137 ━━━━━━━━━━━━━━━━━━━━ 9s 60ms/step - loss: 2.8458
Epoch 14/50
137/137 ━━━━━━━━━━━━━━━━━━━━ 9s 65ms/step - loss: 2.7923
Epoch 15/50
137/137 ━━━━━━━━━━━━━━━━━

In [15]:
def generate_name(model, seed, length=10):
    pattern = [char_to_int[char] for char in seed]
    generated_name = seed
    for _ in range(length):
        x_input = np.reshape(pattern, (1, len(pattern), 1)) / float(len(chars))
        prediction = model.predict(x_input, verbose=0)
        index = np.argmax(prediction)
        result = int_to_char[index]
        generated_name += result.lower()
        pattern.append(index)
        pattern = pattern[1:]
    return generated_name

seed = "Rah"  #Starting seed
new_name = generate_name(model, seed, length=7)
print(new_name)


Rahdraddra


In [16]:
def generate_name_with_temp(model, seed, length=10, temperature=1.0):
    pattern = [char_to_int[char] for char in seed]
    generated_name = seed
    for _ in range(length):
        x_input = np.reshape(pattern, (1, len(pattern), 1)) / float(len(chars))
        prediction = model.predict(x_input, verbose=0)[0]
        prediction = np.asarray(prediction).astype('float64')
        prediction = np.log(prediction + 1e-6) / temperature
        prediction = np.exp(prediction) / np.sum(np.exp(prediction))
        index = np.random.choice(range(len(chars)), p=prediction)
        result = int_to_char[index]
        generated_name += result.lower()
        pattern.append(index)
        pattern = pattern[1:]
    return generated_name

new_name_temp = generate_name_with_temp(model, seed, length=7, temperature=0.5)
print(new_name_temp)


Rahpadishn


In [14]:
model.save("name_generator_model.keras")